## PySpark Streaming

Voorheen hebben we gemerkt met een bestaande, volledige dataset die reeds aanwezig was in de cluster.
Echter is er vaak een nood aan data binnen te halen van verscheidene databronnen, deze om te zetten naar een bruikbaar formaat en te bewaren in een datawarehouse.
Dit is exact wat er nodig is voor het ETL principe of Extract-Transform-Load.
Een belangrijk onderdeel hiervan is de Pyspark Streaming module.
De documentatie van deze module kan [hier](https://spark.apache.org/docs/latest/streaming-programming-guide.html) gevonden worden.
Het wordcount example kan ook hierin geschreven worden, namelijk:

In [3]:
%%file networkwordcount.py
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
# tip: werk met scripts die in bestanden gaan geplaatst worden indien je werkt met streaming
# maakt het makkelijker om streams te stoppen
# deze bestanden moeten standalone zijn: maak de sparkcontext opnieuw aan + zorg voor de nodige imports

# Create a local StreamingContext with two working thread and batch interval of 5 second
sc = SparkContext("local[2]", "networkwordcount")
sc.setLogLevel("ERROR") # reduce spam of logging
ssc = StreamingContext(sc, 5) # om de 5 seconden

lines = ssc.socketTextStream('localhost', 19999) # de poort waarop deze applicatie luistert naar binnenkomende berichten
words = lines.flatMap(lambda line: line.split())
pairs = words.map(lambda x: (x,1))
counts = pairs.reduceByKey(lambda x,y : x+y)

counts.pprint()

ssc.start()
ssc.awaitTermination() # zorg ervoor dat de applicatie actief blijft

Overwriting networkwordcount.py


Om nu de werking te starten, open in jupyterlab een terminal.
Dit is een tool die ons toelaat om tekst te vesturen naar een locale netwerkpoort om zo een tekststroom na te bootsen.
Voer daarna het volgende commando uit:
```console
    nc -lk 19999
```

Het wordcount programma kan nu gestart worden door het volgende commando uit te voeren

In [4]:
!python3 networkwordcount.py

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/20 12:09:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/usr/local/lib/python3.10/dist-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(
-------------------------------------------                                     
Time: 2025-05-20 12:10:00
-------------------------------------------

-------------------------------------------                                     
Time: 2025-05-20 12:10:05
-------------------------------------------
('world', 1)
('ben', 1)
('hello', 1)
('ik', 1)
('jens', 1)

-------------------------------------------                                     
Time: 2025-05-20 12:10:10
-------------------------------------------

---------------------------------

## StreamingContext in plaats van SparkContext

Net zoals de context moet er een streaming context aangemaakt worden voor je de streaming api kan gebruiken.
Een aantal belangrijke punten om te onthouden zijn:
* Eens de context gestart is kan er geen nieuwe code toegevoegd worden
* Eens de context gestopt is kan de context niet opnieuw gestart worden. 
* Er kan maar 1 context tegelijkertijd actief zijn
* De spark context kan hergebruikt worden zolang de streaming context gestopt is voor er een nieuwe streamingcontext aangemaakt wordt.

## DStreams of Discretized Streams

De basis abstractielaag gebruikt en voorzien door Spark Streaming.
Dit stelt een continue datastroom voor die afkomstig is van de inputbron of de verwerkte datastroom van de transform stap.
Deze stream stelt een continue tijdreeks voor van RDD's.
Elk van deze RDD's stelt de ontvangen data tijdens een interval voor.

Deze steams kunnen van verscheidene bronnen komen. 
Indien je een niet-standaard geincludeerde bron wil gebruiken moet je een eigen **receiver** schrijven om de data van de bron op te halen.
Meer informatie over deze procedure vind je [hier](https://spark.apache.org/docs/latest/streaming-custom-receivers.html)

Belangrijk om te onthouden dat het aantal beschikbare cores groter moet zijn dan het aantal receivers/gebruikte databronnen.
Anders beschikt spark/de cluster niet over voldoende rekencapaciteiten om alles parallel uit te voeren.

**Beschikbare transformaties**

De meeste zaken die op een DataFrame/RDD uitgevoerd kunnen worden, kunnen ook op DStreams uitgevoerd worden. 
Een aantal operaties die extra aandacht vereisen zijn
* updateStateByKey()
* transform()
* Window operations
* Join operations

**UpdateStateByKey()**

Deze functie maakt het mogelijk om een algemene state bij te houden en up te daten bij het ontvangen van nieuwe informatie.
Voor het bovenstaande wordcount example kan dit bijvoorbeeld de wordcount van de volledige stream zijn ipv per lijn.
Pas nu het wordcount-example aan door deze twee zaken bij te houden.
Meer informatie kan je [hier](https://spark.apache.org/docs/latest/streaming-programming-guide.html#updatestatebykey-operation) vinden.

**Tip:** Het is nodig om checkpointing te configureren voor deze functie kan gebruikt worden (zodat het ergens kan bijgehouden worden). Dit gebeurt door de volgende lijn na het aanmaken van de streaming context te plaatsen:
    
    ssc.checkpoint("checkpoint")

In [18]:
%%file networkwordcount.py
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from datetime import datetime
# tip: werk met scripts die in bestanden gaan geplaatst worden indien je werkt met streaming
# maakt het makkelijker om streams te stoppen
# deze bestanden moeten standalone zijn: maak de sparkcontext opnieuw aan + zorg voor de nodige imports

# Create a local StreamingContext with two working thread and batch interval of 5 second
sc = SparkContext("local[2]", "networkwordcount")
sc.setLogLevel("ERROR") # reduce spam of logging
ssc = StreamingContext(sc, 5) # om de 5 seconden

# ALS JE WERKT MET STATE MOET JE DIT TOEVOEGEN
ssc.checkpoint('checkpoint') # argument is een pad waar de checkpoints bewaard worden

lines = ssc.socketTextStream('localhost', 19999) # de poort waarop deze applicatie luistert naar binnenkomende berichten
words = lines.flatMap(lambda line: line.split())
pairs = words.map(lambda x: (x,datetime.now())) # in het geval we werken met de timestampss
counts = pairs.reduceByKey(lambda x,y : x+y)

# hou een state bij
def updateFunction(new_values, running_count):
    # concept lijkt om het maken van een accumulator
    return sum(new_values) + (running_count or 0) # nieuwe waarde + oude waarde (of 0 als het de eerste is)

# overschrijf de vorige functie
def updateFunction(new_values, timestamp):
    # return de laatste keer dat een woord is gezien geweest
    return datetime.now()

# overschrijf de vorige functie
def updateFunction(new_values, timestamps):
    # return de laatste keer dat een woord is gezien geweest
    if timestamps is None:
        timestamps = []
    timestamps.extend(new_values)
    return timestamps
    
runningCounts = counts.updateStateByKey(updateFunction)

runningCounts.pprint()

ssc.start()
ssc.awaitTermination() # zorg ervoor dat de applicatie actief blijft

Overwriting networkwordcount.py


In [19]:
!python3 networkwordcount.py

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/20 12:24:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/usr/local/lib/python3.10/dist-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(
-------------------------------------------                                     
Time: 2025-05-20 12:24:45
-------------------------------------------
('test', [datetime.datetime(2025, 5, 20, 12, 24, 45, 785881)])
('world', [datetime.datetime(2025, 5, 20, 12, 24, 45, 785898)])

-------------------------------------------                                     
Time: 2025-05-20 12:24:50
-------------------------------------------
('test', [datetime.datetime(2025, 5, 20, 12, 24, 45, 785881)])
('world', [datetime.datetime(2025, 5, 20, 12, 24, 45, 7

**Transform operation**

De transform operations laat je toe om een RDD-to-RDD functie toe te passenop een DStream.
Dit laat je toe om alle RDD operaties toe te passen die niet zouden aangeboden worden door de Stream API
Het is hierbij belangrijk om op te merken dat deze functie elke batch opgeroepen wordt en dus dat het mogelijk is om parameters te wijzigen tussen de verschillende batches (aantal partities, broadcasted variabelen, ...)

    cleanedDStream = wordCounts.transform(lambda rdd: rdd.join(spamInfoRDD).filter(...))

**Window operations**

Een belangrijke eigenschap van streams is ook dat alle informatie binnen een bepaald tijdsvenster belangrijk kan zijn.
Binnen spark streams zijn er een verscheidene WindowOperations om data binnen een bepaald window te aggregeren en te verwerken.
Hieronder staat een voorbeeld om een reduce toe te passen om de 10 seconden op data dat in de laatste 30 seconden is binnengekomen.

    windowedWordCounts = pairs.reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 30, 10)

**Join operations**

Twee streams kunnen gecombineerd worden door middel van de .join() functie.
Dit doet standaard een inner join maar andere mogelijkheden kunnen ook gekozen worden.

## Sending data to external systems

De foreachRDD functie is een krachtige functie dat toegepast wordt op elke RDD dat aangemaakt wordt door een DStream. 
Dit maakt het mogelijk om de data uit te sturen naar externe systemen en zorgt dus voor de Load-stap binnen ETL.
Een simplistische oplossing is als volgt:

    def sendRecord(rdd):
        connection = createNewConnection()  # executed at the driver
        rdd.foreach(lambda record: connection.send(record))
        connection.close()

    dstream.foreachRDD(sendRecord)
    
Bovenstaande gaat niet werken omdat de connectie door de driver aangemaakt wordt.
Deze connectie gaat geserializeerd worden en doorgestuurd naar de nodes maar dit gaat zelden correct lukken.
Connectieproblemen bij het verzenden van data komen bijna steeds voort uit het correct aanmaken op de juiste nodes van de connectie.
Een oplossing hiervoor is het volgende:

    def sendRecord(record):
        connection = createNewConnection()
        connection.send(record)
        connection.close()

    dstream.foreachRDD(lambda rdd: rdd.foreach(sendRecord))
    
Dit gaat correct werken maar is echter suboptimaal omdat in deze code, een nieuwe connectie aangemaakt wordt voor elke rij in de stream wat voor heel veel overhead zorgt.
Een andere mogelijkheid is als volgt

    def sendPartition(iter):
        connection = createNewConnection()
        for record in iter:
            connection.send(record)
        connection.close()

    dstream.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))
    
Dit is reeds beter omdat de connectie reeds gedeeltelijk hergebruikt wordt maar wordt nog steeds herhaadelijk geopend en gesloten.
Dit zorgt nog steeds voor onnodige overhead.
De beste oplossing is door gebruik te maken van een connectionPool() waaruit connectie kunnen hergebruikt worden.
Deze pool maakt automatisch connecties uit en sluit de bestaande connecties enkel indien ze voldoende lang ongebruikt worden.

    def sendPartition(iter):
        # ConnectionPool is a static, lazily initialized pool of connections
        connection = ConnectionPool.getConnection()
        for record in iter:
            connection.send(record)
        # return to the pool for future reuse
        ConnectionPool.returnConnection(connection)

    dstream.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))

dstream.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))

## Checkpoints

Checkpoints is een manier om informatie op te slaan op de cluster om spark fout-tolerant te worden voor crashes van zowel de driver als individuele node.
Dit gebeurt door de nodige informatie op te slaan in een directory.

Checkpointing moet ge-enabled worden wanneer je
* een state wilt bijhouden
* een fout-tolerante applicatie wil

Let wel op dat het geen garantie is dat alle data behouden blijft maar het merendeel zou correct moeten opgevangen worden.

Checkpointing toevoegen aan je applicatie gebeurt door een directory mee te gevan aan de sparkContext waar de checkpoints in een fout-tolerant gedistribueerd opslagsysteem kunnen bijgehouden worden.
Dit gebeurd als volgt

    ssc.checkpoint("checkpoints") 

**Shared variables with checkpoints**

Accumulators en broadcasted variabelen worden niet opgeslagen door het checkpointing systeem in Spark. 
Dit kan opgelost worden door singleton instances te maken zodat ze kunnen geherinstantieerd worden nadat de driver restart na een failure.
Onderstaande code is een voorbeeld van hoe dit uit te voeren in een wordcount example.
Dit voorbeeld maakt gebruik van globals() wat de globale variabelen bijhoudt.
In dit voorbeeld wordt er gebruik gemaakt van een broadcasted array om een lijst mee te geven met beginletters van woorden die genegeerd worden.
Daarnaast wordt een accumulator gebruikt om het totaal aantal genegeerde woorden te tellen.

In [ ]:
%%file networkwordcount.py
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# broadcast variabele met te negeren woorden
def getWordExcludeList(sparkContext):
    if ("wordExcludeList" not in globals()):
        globals()["wordExcludeList"] = sparkContext.broadcast(["h", "v"])
    return globals()["wordExcludeList"]

# accumulator om het aantal genegeerde woorden dat tegengekomen wordt telt
def getDroppedWordsCounter(sparkContext):
    if ("droppedWordsCounter" not in globals()):
        globals()["droppedWordsCounter"] = sparkContext.accumulator(0)
    return globals()["droppedWordsCounter"]

# Create a local StreamingContext with two working thread and batch interval of 5 second
sc = SparkContext("local[2]", "networkwordcount")
sc.setLogLevel("ERROR") # reduce spam of logging
ssc = StreamingContext(sc, 5)
ssc.checkpoint("checkpoint")

lines = ssc.socketTextStream("localhost", 19999)
words = lines.flatMap(lambda line: line.split(" "))
wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)

# de uit te voeren functie
def echo(time, rdd):
    # Get or register the excludeList Broadcast
    excludeList = getWordExcludeList(rdd.context)
    # Get or register the droppedWordsCounter Accumulator
    droppedWordsCounter = getDroppedWordsCounter(rdd.context)
    
    print(excludeList.value)

    # Use excludeList to drop words and use droppedWordsCounter to count them
    def filterFunc(wordCount):
        w = wordCount[0]
        if len(w)>0 and w[0] in excludeList.value:
            droppedWordsCounter.add(wordCount[1])
            return False
        else:
            return True
        
    f = rdd.filter(filterFunc)

    print("# Genegeerde woorden: %d" % droppedWordsCounter.value)
    print("Filtered rdd:", str(f.collect()))

wordCounts.foreachRDD(echo)

ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate


### Oefening

Schrijf een streaming applicatie dat de volgende kenmerken heeft
* Bereken het aantal woorden dat toekomt met een bepaalde lengte: bvb: "Hello world over there" => 1 woord met 4 karakters, 3 woorden met 5 karakters
* Zorg voor checkpoints in een temp_state directory
* Zorg ervoor dat dit berekend worden in sliding windows van 5 seconden dat elke 2 seconden opschuift
* Hou een algemene state bij met het totaal aantal gelezen woorden, gebruik je hiervoor een accumulator of een update state by key?

### Oplossing

In [ ]:
%%file oefening.py

Zijn de resultaten wat je verwacht had?
Indien niet, hou zou je het kunnen oplossen?

In [ ]:
# antwoord

## Structured streaming

Naast het originele streaming systeem van spark gebruikmakende van DStreams, is er ook een [Structured Streaming](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html) variant.
Het grootste verschil is dat terwijl DStreams gebaseerd zijn op RDD's objecten maakt structured streaming gebruik van DataFrames.
Het networkcount dat we eerst aangehaald hadden hierboven ziet er als volgt uit met structured streaming:

In [24]:
%%file structuredNetworkCount.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col

spark = SparkSession.builder.master('local').appName('les').getOrCreate()

lines = spark.readStream.format('socket').option('host', 'localhost').option('port', 19999).load()

# splits in woorden
words = lines.select(explode(split(lines.value, ' ')).alias('word'))
# genereer aantal
wordCounts = words.groupby('word').count()

# output
query = wordCounts.writeStream.outputMode('update').format('console').start()
# forEachBatch
query.awaitTermination()

Overwriting structuredNetworkCount.py


In [25]:
!python3 structuredNetworkCount.py

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/20 13:16:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/20 13:16:10 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
25/05/20 13:16:11 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-43e51973-7344-4e37-a3ae-c60cee3ce52c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/20 13:16:11 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
-------------------------------------------          

Merk op dat we hier de outputMode complete hebben gebruikt.
Het resulterende gedrag is gelijkaardig aan de state van bij DStreams.
Als we echter ook de wordcount per batch willen weten kunnen we gebruik maken van de andere modes.
Onderstaande voorbeeld toont hoe het networkCount voorbeeld na te bootsen (namelijk via de update mode).

In [ ]:
%%file structuredNetworkCount.py

### Oefening

Gebruik de informatie uit [deze link](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html) en hermaak de vorige oefening van DStreams.
Aangezien structured streaming een globaal dataframe bijhoudt moet er een timestamp toegevoegd worden om het te kunnen verdelen in windows.
Dit moet gedaan worden in de source dus kunnen we hier met deze data geen windows bestuderen.

In [ ]:
%%file oefeningStructured.py